## Biblioteki

In [41]:
import json
import os
import requests
from bs4 import BeautifulSoup

In [68]:
product_id = "11740014"

#url = f"https://www.ceneo.pl/114700014" #https://www.ceneo.pl/114700014
url = f"https://www.ceneo.pl/{product_id}/opinie-1"
response = requests.get(url)
response.status_code

200

# Ekstrakcja składowych pojedynczych opinii
| Składowa | Selektor | Zmienna |
|----------|----------|---------|
| id opinii| ["data-entry-id"] | opinion_id|
| autor    | span.user-post__author-name |author |
| rekomendacja| span.user-post__author-recomendation > em |recommendation|
| gwiazdki | span.user-post__score-count| rating|
| treść    | div.user-post__text|content | 
| lista zalet |div.review-feature__title review-feature__title--positives | pros|
| lista wad | div.review-feature__title review-feature__title--negatives| cons|
| dla ilu przydatna | span[id^="vote-yes"]| useful|
| dla ilu nieprzydatna| span[id^="vote-no"]| useless|
| data wystawienia | span.user-post__published > time:nth-child(1)["datetime"]| post_date|
|data zakupu | span.user-post__published > time:nth-child(2)["datetime"] | purchase_rate |

In [70]:
single_opinion = {
"opinion_id" : opinion['data-entry-id'],
"author": opinion.select_one("span.user-post__author-name").get_text().strip(),## wyciągnie zawartość znacznikow rekurencyjne #.text jako alternatywa do get_text() 
#.strip() pozbywamy się białych znaków
"recommendation" : opinion.select_one('span.user-post__author-recomendation > em'),
"rating" : opinion.select_one("span.user-post__score-count"),
"content" : opinion.select_one("div.user-post__text").get_text().strip(),
"pros" : [p.get_text().strip() for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
"cons" : [c.get_text().strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
"useful" : opinion.select_one('span[id^="vote-yes"]'),
"useless" : opinion.select_one('span[id^="vote-no"]'),
"post_date" : opinion.select_one('span.user-post__published > time:nth-child(1)')["datetime"].strip(),
"purchase_date" : opinion.select_one('span.user-post__published > time:nth-child(2)')["datetime"].strip(),
}



In [75]:
all_opinions = []
while(url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")
    for opinion in opinions:
        try:
            single_opinion = {
                "opinion_id": opinion['data-entry-id'],
                "author": opinion.select_one("span.user-post__author-name").get_text().strip(),## wyciągnie zawartość znacznikow rekurencyjne #.text jako alternatywa do get_text() 
                #.strip() pozbywamy się białych znaków
                "recommendation": opinion.select_one('span.user-post__author-recomendation > em'),
                "rating": opinion.select_one("span.user-post__score-count"),
                "content": opinion.select_one("div.user-post__text").get_text().strip(),
                "pros": [p.get_text().strip() for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
                "cons": [c.get_text().strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
                "useful": opinion.select_one('span[id^="vote-yes"]'),
                "useless": opinion.select_one('span[id^="vote-no"]'),
                "post_date": opinion.select_one('span.user-post__published > time:nth-child(1)')["datetime"].strip(),
                "purchase_date": opinion.select_one('span.user-post__published > time:nth-child(2)')["datetime"].strip()
            }
            all_opinions.append(single_opinion)
        except (AttributeError, TypeError):
            pass
    try:
        url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"]
    except TypeError:
        url = None

In [72]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open(f"opinions/{product_id}.json","w",encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii = False) #narodowe znaki diakrytyczne będą się poprawnie wyświetlać

In [60]:
len(opinions)

10